# Capstone Notebook
This notebook will be used for the capstone project for the IBM Data Science Professional Certificate

In [1]:
import pandas as pd
import numpy as np

In [2]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

In [3]:
from bs4 import BeautifulSoup
import urllib.request
import re

In [4]:
#create beautiful soup object for the given url
try:
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",None,headers) #The assembled request
    html = urllib.request.urlopen(request)
except:
    print("Invalid URL, cannot open")
            #return
soup = BeautifulSoup(html, 'lxml')  

In [5]:
#find the table and separate into body and head
table=soup.find_all("table", attrs={"class": "wikitable"})
table1 = table[0]
# the head will form our column names
body = table1.find_all("tr")
# Head values (Column names) are the first items of the body list
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

In [6]:
#load in the headings of the table
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


In [7]:
#add in the rows of the table
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa) #add to list containing row entry
    if row[1]=='Not assigned': #no borough assigned
        continue;
    else:
        if row[2]=='Not assigned': #borough assigned, but no neighborhood
            row[2]=row[1]
        all_rows.append(row)



In [8]:
df = pd.DataFrame(data=all_rows,columns=headings)

In [9]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park Harbourfront
3,M6A,North York,Lawrence Manor Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue Humber Valley Village
6,M1B,Scarborough,Malvern Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill Woodbine Gardens
9,M5B,Downtown Toronto,Garden District Ryerson


1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
4. If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
6. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
7. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df.shape

(103, 3)

## Using csv file to create dataframe with coordinates

In [11]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_toronto=pd.merge(df, df_data_1, on='Postal Code', how='outer')
df_toronto.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
df_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue Humber Valley Village,43.667856,-79.532242
6,M1B,Scarborough,Malvern Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,Garden District Ryerson,43.657162,-79.378937


## Exploration and Clustering of Neighborhoods in Toronto

In [17]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                    -failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|1_73_pypy|3_73_pypy|4_73_pypy|5_73_pypy|2_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python 

ModuleNotFoundError: No module named 'folium'

In [ ]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
toronto_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [ ]:
# The code was removed by Watson Studio for sharing.